# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,289.92,59,56,6.80,AU,1706983366
1,1,iqaluit,63.7506,-68.5145,248.00,69,20,5.14,CA,1706983445
2,2,vila velha,-20.3297,-40.2925,304.12,80,75,4.63,BR,1706983276
3,3,yellowknife,62.4560,-114.3525,255.60,96,100,7.20,CA,1706983308
4,4,west island,-12.1568,96.8225,301.14,83,40,4.63,CC,1706983508


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", size = "Humidity", tiles = "OSM")
                                     

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 308) & (city_data_df["Max Temp"] > 288) & (city_data_df["Humidity"] < 20)]

# Drop any rows with null values
my_cities_df.dropna()

# Display sample data
my_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
196,196,broken hill,-31.9500,141.4333,300.76,17,98,4.14,AU,1706983648
213,213,kuraymah,18.5500,31.8500,291.28,18,52,4.84,SD,1706983659
226,226,ayorou,14.7308,0.9174,299.24,13,55,4.77,NE,1706983668
265,265,sarmiento,-45.5882,-69.0700,291.67,17,9,2.74,AR,1706983695
285,285,taoudenni,22.6783,-3.9836,296.13,9,5,5.13,ML,1706983708
347,347,pico truncado,-46.7949,-67.9573,291.11,14,6,2.45,AR,1706983751
363,363,wamba,8.9333,8.6000,301.09,16,16,1.58,NG,1706983763
373,373,wad az zaki,14.4619,32.2065,294.89,14,23,5.28,SD,1706983770
384,384,hidalgo,20.5000,-99.0000,295.84,15,0,7.64,MX,1706983777
434,434,tindouf,27.6711,-8.1474,290.35,8,23,4.42,DZ,1706983812


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=' '

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
196,broken hill,AU,-31.9500,141.4333,17,
213,kuraymah,SD,18.5500,31.8500,18,
226,ayorou,NE,14.7308,0.9174,13,
265,sarmiento,AR,-45.5882,-69.0700,17,
285,taoudenni,ML,22.6783,-3.9836,9,
347,pico truncado,AR,-46.7949,-67.9573,14,
363,wamba,NG,8.9333,8.6000,16,
373,wad az zaki,SD,14.4619,32.2065,14,
384,hidalgo,MX,20.5000,-99.0000,15,
434,tindouf,DZ,27.6711,-8.1474,8,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
broken hill - nearest hotel: Ibis Styles
kuraymah - nearest hotel: Hotel al Nasser
ayorou - nearest hotel: Hotel Aminokal
sarmiento - nearest hotel: Ismar
taoudenni - nearest hotel: No hotel found
pico truncado - nearest hotel: No hotel found
wamba - nearest hotel: No hotel found
wad az zaki - nearest hotel: No hotel found
hidalgo - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
zouerate - nearest hotel: فندق تيرس زمور
tidjikja - nearest hotel: No hotel found
kiffa - nearest hotel: فندق كيفة
ouadda - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
196,broken hill,AU,-31.9500,141.4333,17,Ibis Styles
213,kuraymah,SD,18.5500,31.8500,18,Hotel al Nasser
226,ayorou,NE,14.7308,0.9174,13,Hotel Aminokal
265,sarmiento,AR,-45.5882,-69.0700,17,Ismar
285,taoudenni,ML,22.6783,-3.9836,9,No hotel found
347,pico truncado,AR,-46.7949,-67.9573,14,No hotel found
363,wamba,NG,8.9333,8.6000,16,No hotel found
373,wad az zaki,SD,14.4619,32.2065,14,No hotel found
384,hidalgo,MX,20.5000,-99.0000,15,No hotel found
434,tindouf,DZ,27.6711,-8.1474,8,محمد بوسبي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, color = "City", size = "Humidity", tiles = "OSM", hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)